In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt, mpld3
import thermoDataGrabber as DG
import CoolProp.CoolProp as CP
from cytoolz import get
%matplotlib notebook
mpld3.enable_notebook()
#print(plt.style.available)
plt.style.use(["seaborn-talk","seaborn-notebook","seaborn-paper"])
rescale = lambda old, new: lambda x: (new[1] - new[0]) / (old[1] - old[0])*(x - old[0]) + new[0]

In [ ]:
myfluid = DG.ThermoFluid("Water", "T", "P", "U", [217, 217], "viridis")
print(myfluid.data.shape)

In [ ]:
buffer = {}
derivatives = {"d(U)/d(T)|P": lambda state: CP.PropsSI("d(U)/d(T)|P", 'P', state["P"], 'T', state["T"], myfluid.fluid),
               "d(U)/d(P)|T": lambda state: CP.PropsSI("d(U)/d(P)|T", 'P', state["P"], 'T', state["T"], myfluid.fluid),
              }

for key in derivatives:
    buffer[key] = []
               
for index, row in myfluid.data.iterrows():
    for key in derivatives:
        get(key, buffer).append(derivatives[key](row))

In [ ]:
for key in derivatives:
    myfluid.data[key] = pd.Series(get(key, buffer), index=myfluid.data.index)
    
rescaled = myfluid.copy()

In [ ]:
tmax = max(myfluid.data["T"])
tmin = min(myfluid.data["T"])
trange = [tmin, tmax]
pmax = max(myfluid.data["P"])
pmin = min(myfluid.data["P"])
prange = [pmin, pmax]
umax = max(myfluid.data["U"])
umin = min(myfluid.data["U"])
urange = [umin, umax]
dmax1 = max(myfluid.data["d(U)/d(T)|P"])
dmin1 = min(myfluid.data["d(U)/d(T)|P"])
drange1 = [dmin1, dmax1]
dmax2 = max(myfluid.data["d(U)/d(P)|T"])
dmin2 = min(myfluid.data["d(U)/d(P)|T"])
drange2 = [dmin2, dmax2]

In [ ]:
rescaled.data["T"] = rescaled.data["T"].apply(rescale(trange, [0,216]))
rescaled.data["P"] = rescaled.data["P"].apply(rescale(prange, [0,216]))
rescaled.data["U"] = rescaled.data["U"].apply(rescale(urange, [15, 152]))

In [ ]:
# for i in range(4):
#     dU = 2 + i
#     uSel = 25 + 25*i
#     rescaled.ix[np.abs(rescaled["U"] - uSel) <= 1, 'U'] = rescaled.ix[np.abs(rescaled["U"] - uSel) <= 1 , "U"] + dU
#     rescaled.ix[np.abs(rescaled["U"] - uSel - 12.5) <= 1.5, 'U'] = rescaled.ix[np.abs(rescaled["U"] - uSel - 12.5) <= 1.5 , "U"] - dU

rescaled.data.ix[np.abs(rescaled.data["d(U)/d(T)|P"] - 10000) <= 2500, 'U'] = rescaled.data.ix[np.abs(rescaled.data["d(U)/d(T)|P"] - 10000) <= 2500, 'U'] + 10

In [ ]:
myfluid2 = rescaled.copy()
DG.fluid_plot(myfluid2)

In [ ]:
rescaled.data[np.abs(rescaled.data["d(U)/d(T)|P"] - 10000) <= 2500]